## 1. Preprocess documents

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./rag_data/policy.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

In [ ]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200, chunk_overlap=50, add_start_index=True
)
doc_splits = text_splitter.split_documents(docs)

print(len(doc_splits))

## 2. Create a retriever tool

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_classic.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_service_policy",
    "Search and return information about the '모두의주차장' policy document.",
)

In [ ]:
retriever_tool.invoke({"query": "부설주차장이 뭐야?"})


## 3. Generate query

In [ ]:
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model

response_model = init_chat_model("gpt-5-nano", temperature=0)


def generate_query_or_respond(state: MessagesState):
    """Call the model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply respond to the user.
    """
    response = (
        response_model
        .bind_tools([retriever_tool]).invoke(state["messages"])  
    )
    return {"messages": [response]}

In [ ]:
input = {"messages": [{"role": "user", "content": "hello!"}]}
generate_query_or_respond(input)["messages"][-1].pretty_print()

In [ ]:
input = {
    "messages": [
        {
            "role": "user",
            "content": "공유주차장에서 구획제와 구간제의 차이점이 뭐야?",
        }
    ]
}
generate_query_or_respond(input)["messages"][-1].pretty_print()


## 4. Grade documents

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

GRADE_PROMPT = (
    "You are a grader assessing relevance of a retrieved document to a user question. \n "
    "Here is the retrieved document: \n\n {context} \n\n"
    "Here is the user question: {question} \n"
    "If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n"
    "Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."
)


class GradeDocuments(BaseModel):  
    """Grade documents using a binary score for relevance check."""

    binary_score: str = Field(
        description="Relevance score: 'yes' if relevant, or 'no' if not relevant"
    )


grader_model = init_chat_model("gpt-5-nano", temperature=0)


def grade_documents(
    state: MessagesState,
) -> Literal["generate_answer", "rewrite_question"]:
    """Determine whether the retrieved documents are relevant to the question."""
    question = state["messages"][0].content
    context = state["messages"][-1].content

    prompt = GRADE_PROMPT.format(question=question, context=context)
    response = (
        grader_model
        .with_structured_output(GradeDocuments).invoke(  
            [{"role": "user", "content": prompt}]
        )
    )
    score = response.binary_score

    if score == "yes":
        return "generate_answer"
    else:
        return "rewrite_question"

In [ ]:
from langchain_core.messages import convert_to_messages

input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "부설주차장이 뭐야?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_service_policy",
                        "args": {"query": "부설주차장은 무엇인가"},
                    }
                ],
            },
            {"role": "tool", "content": "아마노", "tool_call_id": "1"},
        ]
    )
}
grade_documents(input)

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "공유주차장에서 구획제와 구간제의 차이점이 뭐야?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_service_policy",
                        "args": {"query": "구획제 구간제"},
                    }
                ],
            },
            {
                "role": "tool",
                "content": "'구획 / 구간\n구획제\n구획  번호가  지정된  자리에  배정  받은  차량만  이용할  수  있는  제도이다 . ‘구획주차면 ʼ 을  구\n\n구간제\n정해진  구간  내에서  배정자들이  자유롭게  주차할  수  있는  제도이다 . 여러  개의  주차면을  묶\n\n음 )\n💡실제  구간제로  운영되는  곳인지 , 행정  구분상  구간제로  표기해둔  것인지를  알기  위해\n\n서는  Admin 상에서  해당  구간  번호를  클릭  해서  구간인지  구획인지  확인  해야  한다 .\n어 플  상 에 서 는  6 - 7 - 2 P 로  표 기 되 지 만'",
                "tool_call_id": "1",
            },
        ]
    )
}
grade_documents(input)

In [ ]:
retriever_tool.invoke({"query": "구획제 구간제"})


## 5. Rewrite question

In [ ]:
from langchain.messages import HumanMessage

REWRITE_PROMPT = (
    "Look at the input and try to reason about the underlying semantic intent / meaning.\n"
    "Here is the initial question:"
    "\n ------- \n"
    "{question}"
    "\n ------- \n"
    "Formulate an improved question:"
)


def rewrite_question(state: MessagesState):
    """Rewrite the original user question."""
    messages = state["messages"]
    question = messages[0].content
    prompt = REWRITE_PROMPT.format(question=question)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [HumanMessage(content=response.content)]}

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "부설주차장이 뭐야?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_service_policy",
                        "args": {"query": "부설주차장은 무엇인가"},
                    }
                ],
            },
            {"role": "tool", "content": "아마노", "tool_call_id": "1"},
        ]
    )
}

response = rewrite_question(input)
print(response["messages"][-1].content)

## 6. Generate an answer

In [ ]:
GENERATE_PROMPT = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, just say that you don't know. "
    "Use three sentences maximum and keep the answer concise.\n"
    "Question: {question} \n"
    "Context: {context}"
)


def generate_answer(state: MessagesState):
    """Generate an answer."""
    question = state["messages"][0].content
    context = state["messages"][-1].content
    prompt = GENERATE_PROMPT.format(question=question, context=context)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [response]}

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "공유주차장에서 구획제와 구간제의 차이점이 뭐야?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_service_policy",
                        "args": {"query": "구획제 구간제"},
                    }
                ],
            },
            {
                "role": "tool",
                "content": "'구획 / 구간\n구획제\n구획  번호가  지정된  자리에  배정  받은  차량만  이용할  수  있는  제도이다 . ‘구획주차면 ʼ 을  구\n\n구간제\n정해진  구간  내에서  배정자들이  자유롭게  주차할  수  있는  제도이다 . 여러  개의  주차면을  묶\n\n음 )\n💡실제  구간제로  운영되는  곳인지 , 행정  구분상  구간제로  표기해둔  것인지를  알기  위해\n\n서는  Admin 상에서  해당  구간  번호를  클릭  해서  구간인지  구획인지  확인  해야  한다 .\n어 플  상 에 서 는  6 - 7 - 2 P 로  표 기 되 지 만'",
                "tool_call_id": "1",
            },
        ]
    )
}

response = generate_answer(input)
response["messages"][-1].pretty_print()

7. Assemble the graph

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

workflow = StateGraph(MessagesState)

# Define the nodes we will cycle between
workflow.add_node(generate_query_or_respond)
workflow.add_node("retrieve", ToolNode([retriever_tool]))
workflow.add_node(rewrite_question)
workflow.add_node(generate_answer)

workflow.add_edge(START, "generate_query_or_respond")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "generate_query_or_respond",
    # Assess LLM decision (call `retriever_tool` tool or respond to the user)
    tools_condition,
    {
        # Translate the condition outputs to nodes in our graph
        "tools": "retrieve",
        END: END,
    },
)

# Edges taken after the `action` node is called.
workflow.add_conditional_edges(
    "retrieve",
    # Assess agent decision
    grade_documents,
)
workflow.add_edge("generate_answer", END)
workflow.add_edge("rewrite_question", "generate_query_or_respond")

# Compile
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

8. Run the agentic RAG

In [ ]:
for chunk in graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "공유주차장에서 구획제와 구간제의 차이점이 뭐야?",
            }
        ]
    }
):
    for node, update in chunk.items():
        print("Update from node", node)
        update["messages"][-1].pretty_print()
        print("\n\n")